In [ ]:
!pip install pycuda

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

$
\begin{bmatrix}
 a_{11} &  \dots  & a_{1n} \\
 \vdots &  \ddots & \vdots \\
 a_{m1} &  \dots  & a_{mn}
\end{bmatrix}
+
\begin{bmatrix}
 b_{11} & \dots  & b_{1n} \\
 \vdots & \ddots & \vdots \\
 b_{m1} & \dots  & b_{mn}
\end{bmatrix}
=\begin{bmatrix}
 a_{11} + b_{11} & \dots  & a_{1n} + b_{1n} \\
     \vdots      & \ddots &     \vdots      \\
 a_{m1} + b_{m1} & \dots  & a_{mn} + b_{mn}
\end{bmatrix}
$

In [ ]:
def add_mat(A,B):
  dim1,dim2 = A.shape
  C = np.empty_like(A)
  for i in range(dim1):
    for j in range(dim2):
      C[i,j] = A[i,j] + B[i,j]
  return C

#Compléter le kernel

In [ ]:
kernels = SourceModule("""
// first kernel
__global__ void add_mat(float *g_A,float *g_B,float *g_C)
{
   // each thread compute only one C[i,j]=A[i,j]+B[i,j]
   int idx = TODO ;
   g_C[idx] = TODO ;
}
// second kernel
__global__ void add_huge_mat(float *g_A,float *g_B,float *g_C,int width)
{
   // each thread compute  one line of C[i,j]=A[i,j]+B[i,j] for j from 0 to width
   // idx is the matrices line number
   int idx =  TODO ;
   for (int j=0;j<width;j++)
      g_C[width*idx+j] = TODO ;
}
""")

#Initialisation des matrices sur le host

In [ ]:
h_a = np.random.randn(32,32)
h_b = np.random.randn(32,32)
h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)

#Lecture du kernel

In [ ]:
kadd_mat      = kernels.get_function("add_mat")

#Compléter les allocations mémoire et l'écriture sur le device

In [ ]:
g_A = cuda.mem_alloc(...)
g_B = cuda.mem_alloc(...)
g_C = cuda.mem_alloc(...)
cuda.memcpy_htod(...,...)
cuda.memcpy_htod(...,...)


#Appeler le kernel avec le bon block

In [ ]:
kadd_mat(g_A,g_B,g_C,block=(...,...,...))

#Récupération du résultat sur le host

In [ ]:
h_c = np.empty_like(h_a)
cuda.memcpy_dtoh(h_c,g_C)

#Libération de la mémoire sur le device

In [ ]:
g_A.free()
g_B.free()
g_C.free()

#Affichage résultat

In [ ]:
np.array_equal(h_a+h_b,h_c)

#Adapter le kernel add_huge_mat afin de prendre en charge les matrices suivantes

In [ ]:
h_HA = np.random.randn(1024,1024)
h_HB = np.random.randn(1024,1024)
h_HA = h_HA.astype(np.float32)
h_HB = h_HB.astype(np.float32)

In [ ]:
g_HA = cuda.mem_alloc(...)
g_HB = cuda.mem_alloc(...)
g_HC = cuda.mem_alloc(...)

In [ ]:
cuda.memcpy_htod(...,...)
cuda.memcpy_htod(g_HB,h_HB)

In [ ]:
dim1,dim2=h_HA.shape
# chaque thread prend en charge width operations dans le cas où l'opération
#   tombe juste width = nombre d'éléments par matrice / nombre de threads
width=np.int32((dim1*dim2)/(32*32))

In [ ]:
kadd_huge_mat = kernels.get_function("add_huge_mat")

In [ ]:
kadd_huge_mat(g_HA,g_HB,g_HC,width,block=(32,32,1))

In [ ]:
h_HC = np.empty_like(h_HA)
cuda.memcpy_dtoh(h_HC,g_HC)

In [ ]:
g_HA.free()
g_HB.free()
g_HC.free()